In [2]:
from sentence_transformers import SentenceTransformer
import gradio as gr
import pandas as pd


## TO DO 
## change the model and the dataset 

model = SentenceTransformer('all-MiniLM-L12-v2')

sds = pd.read_csv("data/small_dataset.csv")

embeddings = model.encode(sds['lyrics'])
sds['embeddings'] = list(embeddings)

2022-12-07 22:55:49.810798: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-07 22:55:49.810819: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [35]:
def relevance_scores(query_embed):
    scores = [cosine_similarity(query_embed, v2) for v2 in sds['embeddings']]
    scores = pd.Series(scores)
    return(scores)


def semantic_search(query_sentence, df = sds, return_top = False):
    query_embed = model.encode(query_sentence)
    scores = relevance_scores(query_embed)
    df['scores'] = scores
    sorted_df = df.sort_values(by = 'scores', ascending = False)
    if return_top == False:
        sorted_df['scores'] = round(sorted_df['scores'],3)
        return sorted_df[['title','artist','scores']].head(3)
    else:
        return sorted_df.iloc[0]['lyrics'][:200]
    
def cosine_similarity(v1, v2):
    d = np.dot(v1, v2)
    cos_theta = d / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return(cos_theta)

In [36]:
semantic_search("i'm pleased you are doing well after we left each other")

,title,artist,scores
1313,​happier,Olivia Rodrigo,0.401
694,Let Me Love You,Ariana Grande,0.395
1375,​good 4 u,Olivia Rodrigo,0.390


In [37]:
print(semantic_search("i'm pleased you are doing well after we left each other", return_top = True))


We broke up a month ago
Your friends are mine, you know I know
You've moved on, found someone new
One more girl who brings out the better in you
And I thought my heart was detached
From all the sunli


In [40]:
## To do 

# here we load our model 

def get_recom(query): 
    return  semantic_search(query), semantic_search(query, return_top = True) 

iface = gr.Interface(    
    title = 'Enjoy our recommendations', 
                     description = 'Do you think we can guess what you like?', 
                    fn=get_recom, 
                     inputs=gr.Textbox(lines=4, placeholder="Enter ur query...", label = 'Query',
                                       show_label = True),
    outputs = [gr.Dataframe(label = "Top 3 songs", show_label = True), 
               gr.Text(label = 'A glimpse of the closest match', show_label = True)]
    ,live = False,
    interpretation="default",
)


iface.launch(share = True)

Running on local URL:  http://127.0.0.1:7872/
Running on public URL: https://20c32e9d83153dfa.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://www.huggingface.co/spaces)


(<gradio.routes.App at 0x7f47ceaed7c0>,
 'http://127.0.0.1:7872/',
 'https://20c32e9d83153dfa.gradio.app')